# Transformer 실습

이번 실습에서는 감정 분석 task에 RNN 대신 Transformer를 구현하여 적용해 볼 것입니다.
Library import나 dataloader 생성은 RNN 실습 때와 똑같기 때문에 설명은 넘어가도록 하겠습니다.

In [24]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["TORCH_USE_CUDA_DSA"] = '1'

In [25]:
!pip install datasets

In [26]:
pip install sacremoses

In [27]:
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torch.utils.data import DataLoader
from transformers import BertTokenizerFast
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)


ds = load_dataset("stanfordnlp/imdb")
tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-uncased')

    # 학습 로직


from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 400  # 최대 시퀀스 길이 설정

from torch.nn.utils.rnn import pad_sequence
import torch

from torch.nn.utils.rnn import pad_sequence
import torch

def collate_fn(batch):
    texts, labels = [], []
    max_len = 400  # 최대 길이 설정

    for item in batch:
        encoded_text = tokenizer.encode_plus(
            item['text'],
            add_special_tokens=True,
            return_tensors='pt',
            padding='max_length',  # 패딩 적용
            truncation=True,  # 자르기 적용
            max_length=max_len  # 최대 길이 설정
        )
        input_ids = encoded_text['input_ids'].squeeze(0)

        # 입력은 전체 시퀀스를 사용하고, 레이블은 다음 토큰을 사용합니다.
        texts.append(input_ids[:-1])  # 마지막 토큰을 제외한 나머지를 입력으로 사용
        labels.append(input_ids[1:])  # 첫 번째 토큰을 제외한 나머지를 레이블로 사용

    if not texts or not labels:  # 입력이 비어 있는 경우 기본 값을 추가
        default_value = torch.tensor([tokenizer.pad_token_id], dtype=torch.long)
        texts.append(default_value)
        labels.append(torch.tensor([-100], dtype=torch.long))  # 손실 계산에서 무시될 값

    # 텐서로 변환하고 배치 차원에서 패딩 적용
    texts = pad_sequence(texts, batch_first=True, padding_value=tokenizer.pad_token_id)
    labels = pad_sequence(labels, batch_first=True, padding_value=-100)

    return texts, labels








Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [30]:
train_loader = DataLoader(
    ds['train'], batch_size=32, shuffle=True, collate_fn=collate_fn
)
test_loader = DataLoader(
    ds['test'], batch_size=32, shuffle=False, collate_fn=collate_fn

)

In [31]:
from torch import nn
from math import sqrt


class SelfAttention(nn.Module):
    def __init__(self, input_dim, d_model):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model

        self.wq = nn.Linear(input_dim, d_model)
        self.wk = nn.Linear(input_dim, d_model)
        self.wv = nn.Linear(input_dim, d_model)
        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        q, k, v = self.wq(x), self.wk(x), self.wv(x)
        score = torch.matmul(
            q, k.transpose(-1, -2)
        )  # (B, S, D) * (B, D, S) = (B, S, S)
        score = score / sqrt(self.d_model)

        if mask is not None:
            score = score + (mask * -1e9)

        score = self.softmax(score)
        result = torch.matmul(score, v)
        result = self.dense(result)

        return result

In [53]:
class MultiHeadAttention(nn.Module):

    def __init__(
        self,
        input_dim,
        d_model,
        n_heads,
    ):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.n_heads = n_heads

        assert (
            d_model % n_heads == 0
        ), f"d_model ({d_model}) must be a multiple of n_heads ({n_heads})"

        self.d_prime = self.d_model // self.n_heads

        self.wqkvs = nn.ModuleList(
            [
                nn.ModuleDict(
                    {
                        "wq": nn.Linear(input_dim, self.d_prime),
                        "wk": nn.Linear(input_dim, self.d_prime),
                        "wv": nn.Linear(input_dim, self.d_prime),
                    }
                )
                for _ in range(self.n_heads)
            ]
        )

        self.dense = nn.Linear(d_model, d_model)

        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x, mask):
        results = []
        for wqkv in self.wqkvs:
            q, k, v = wqkv["wq"](x), wqkv["wk"](x), wqkv["wv"](x)
            score = torch.matmul(
                q,
                k.transpose(-1, -2),
            )  # (B, S, D') * (B, D', S) = (B, S, S)
            score = score / sqrt(self.d_prime)

            if mask is not None:
                score = score + (mask * -1e9)

            score = self.softmax(score)
            result = torch.matmul(
                score,  # (B, S, S)
                v,  # (B, S, D')
            )  # (B, S, D')
            results.append(result)
        results = torch.cat(results, dim=-1)  # (B, S, D' * n_heads)
        output = self.dense(results)
        return output

In [54]:
class TransformerLayer(nn.Module):

    def __init__(self, input_dim, d_model, dff, n_heads, dropout_prob=0.3):
        super().__init__()

        self.input_dim = input_dim
        self.d_model = d_model
        self.dff = dff

        self.mha = MultiHeadAttention(input_dim, d_model, n_heads)
        self.ffn = nn.Sequential(
            nn.Linear(d_model, dff), nn.ReLU(), nn.Linear(dff, d_model)
        )
        self.dropout = nn.Dropout(dropout_prob)
        self.layer_norm_mha = nn.LayerNorm(normalized_shape=d_model)
        self.layer_norm_ffn = nn.LayerNorm(normalized_shape=d_model)

    def forward(self, x, mask):
        x1 = self.mha(x, mask)
        x1 = self.dropout(x1)
        x1 = self.layer_norm_mha(x + x1)

        x2 = self.ffn(x1)
        x2 = self.dropout(x2)
        x2 = self.layer_norm_ffn(x1 + x2)

        return x2


In [55]:
import numpy as np


def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, None], np.arange(d_model)[None, :], d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[None, ...]

    return torch.FloatTensor(pos_encoding)


max_len = 400
print(positional_encoding(max_len, 256).shape)

torch.Size([1, 400, 256])


In [56]:
import torch
import torch.nn as nn

class TextClassifier(nn.Module):
    def __init__(self, vocab_size, d_model, n_layers, dff, n_heads, max_len):
        super().__init__()
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.n_layers = n_layers
        self.dff = dff
        self.n_heads = n_heads

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoding = nn.parameter.Parameter(
            positional_encoding(max_len, d_model), requires_grad=False
        )
        self.layers = nn.ModuleList(
            [TransformerLayer(d_model, d_model, dff, n_heads) for _ in range(n_layers)]
        )
        self.classification = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        mask = x == tokenizer.pad_token_id
        mask = mask[:, None, :]
        seq_len = x.shape[1]

        x = self.embedding(x)
        x = x * sqrt(self.d_model)
        x = x + self.pos_encoding[:, :seq_len]

        for layer in self.layers:
            x = layer(x, mask)

        x = self.classification(x)  # 각 위치에서의 출력
        return x  # 로짓 반환, softmax는 손실 계산 시 적용

# 모델 초기화 및 사용 예
model = TextClassifier(
    vocab_size=len(tokenizer),
    d_model=512,
    n_layers=6,
    dff=2048,
    n_heads=8,
    max_len=512
)


In [57]:
from torch.optim import Adam

lr = 0.001
model = model.to('cuda')
loss_fn = nn.CrossEntropyLoss()

optimizer = Adam(model.parameters(), lr=lr)

In [58]:
def accuracy(model, dataloader):
    model.eval()
    total_correct = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to('cuda'), labels.to('cuda')

            outputs = model(inputs)  # 모델의 출력 크기: (batch_size, seq_len, vocab_size)
            predicted = outputs.argmax(dim=2)  # 가장 확률이 높은 단어의 인덱스를 추출

            # 예측과 레이블을 평면화(flatten)하여 전체 정확도를 계산
            total_correct += (predicted == labels).sum().item()
            total_samples += labels.numel()

    acc = total_correct / total_samples if total_samples > 0 else 0
    return acc



In [ ]:
n_epochs = 50
for epoch in range(n_epochs):
    total_loss = 0.
    model.train()

    for inputs, labels in train_loader:
        inputs, labels = inputs.to('cuda'), labels.to('cuda')

        optimizer.zero_grad()
        preds = model(inputs)  # 예상 크기: (batch_size, seq_len, vocab_size)

        # 손실 계산 및 역전파
        loss = loss_fn(preds.transpose(1, 2), labels)  # labels 크기: (batch_size, seq_len)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Total Loss: {total_loss}")

    with torch.no_grad():
        model.eval()
        train_acc = accuracy(model, train_loader)
        test_acc = accuracy(model, test_loader)
        print(f"=========> Train acc: {train_acc:.3f} | Test acc: {test_acc:.3f}")



Epoch 1 | Total Loss: 3828.3346264362335
=========> Train acc: 0.365 | Test acc: 0.373
Epoch 2 | Total Loss: 3823.7562959194183
=========> Train acc: 0.365 | Test acc: 0.373
Epoch 3 | Total Loss: 3775.4418153762817
=========> Train acc: 0.365 | Test acc: 0.373
Epoch 4 | Total Loss: 3767.5583319664
=========> Train acc: 0.365 | Test acc: 0.373
